<a href="https://colab.research.google.com/github/Kai4xb/E6908_project_ky2513/blob/main/LSTM_Shot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup Python Environment 

In [ ]:
%%capture
# Setup environment
# !apt-get -qq install xxd
!sudo apt-get update -y
!sudo apt-get install -y xxd
!pip install pandas numpy matplotlib
%tensorflow_version 2.x
!pip install tensorflow
!pip install h5py


In [ ]:
 ! python --version 
 print(f"TensorFlow version = {tf.__version__}\n")

Python 3.10.11
TensorFlow version = 2.12.0



In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import files
uploaded = files.upload()
correct_shots = pd.read_csv("test1.csv", names=['ax', 'ay', 'az', 'gx', 'gy', 'gz'])
uploaded = files.upload()
incorrect_shots = pd.read_csv("test2.csv", names=['ax', 'ay', 'az', 'gx', 'gy', 'gz'])
print("correct shape:", correct_shots.shape)
print("incorrect shape:", incorrect_shots.shape)

Saving test1.csv to test1.csv


Saving test2.csv to test2.csv
correct shape: (1727, 6)
incorrect shape: (946, 6)


In [ ]:
window_size = 100
step_size = 10

def create_windows(data, window_size, step_size):
    windows = []
    labels = []
    for i in range(0, len(data) - window_size + 1, step_size):
        windows.append(data.iloc[i:i+window_size, :-1].values)
        labels.append(data.iloc[i+window_size-1, -1])
    return np.array(windows), np.array(labels)

correct_windows, correct_labels = create_windows(correct_shots, window_size, step_size)
incorrect_windows, incorrect_labels = create_windows(incorrect_shots, window_size, step_size)

print("correct_windows shape:", correct_windows.shape)
print("correct_labels shape:", correct_labels.shape)
print("incorrect_windows shape:", incorrect_windows.shape)
print("incorrect_labels shape:", incorrect_labels.shape)

data_X = np.vstack((correct_windows, incorrect_windows))
data_y = np.hstack((correct_labels, incorrect_labels))

print("data_X shape:", data_X.shape)
print("data_y shape:", data_y.shape)

train_data, test_data, train_labels, test_labels = train_test_split(data_X, data_y, test_size=0.2, random_state=42)
train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size=0.25, random_state=42)

train_X, train_y = extract_features_and_labels(train_data, train_labels)
val_X, val_y = extract_features_and_labels(val_data, val_labels)
test_X, test_y = extract_features_and_labels(test_data, test_labels)



correct_windows shape: (161, 100, 6)
correct_labels shape: (161,)
incorrect_windows shape: (81, 100, 6)
incorrect_labels shape: (81,)
data_X shape: (242, 100, 6)
data_y shape: (242,)


In [ ]:
def extract_features_and_labels(data, labels):
    return data, labels

train_X, train_y = extract_features_and_labels(train_data, train_labels)
val_X, val_y = extract_features_and_labels(val_data, val_labels)
test_X, test_y = extract_features_and_labels(test_data, test_labels)


In [ ]:
model = Sequential([
    LSTM(32, input_shape=(100, 6), return_sequences=True),
    Dropout(0.2),
    LSTM(16),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])


history = model.fit(train_X, train_y, batch_size=32, epochs=30, validation_data=(val_X, val_y))

print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")

Epoch 1/30
5/5 [==============================] - 6s 198ms/step - loss: 0.6658 - accuracy: 0.5833 - val_loss: 0.6578 - val_accuracy: 0.5918
Epoch 2/30
5/5 [==============================] - 0s 21ms/step - loss: 0.6155 - accuracy: 0.7222 - val_loss: 0.6171 - val_accuracy: 0.7551
Epoch 3/30
5/5 [==============================] - 0s 21ms/step - loss: 0.5932 - accuracy: 0.7639 - val_loss: 0.5805 - val_accuracy: 0.7755
Epoch 4/30
5/5 [==============================] - 0s 25ms/step - loss: 0.5607 - accuracy: 0.7778 - val_loss: 0.5486 - val_accuracy: 0.7959
Epoch 5/30
5/5 [==============================] - 0s 21ms/step - loss: 0.5289 - accuracy: 0.8125 - val_loss: 0.5164 - val_accuracy: 0.8163
Epoch 6/30
5/5 [==============================] - 0s 19ms/step - loss: 0.5021 - accuracy: 0.7986 - val_loss: 0.4807 - val_accuracy: 0.8367
Epoch 7/30
5/5 [==============================] - 0s 20ms/step - loss: 0.4701 - accuracy: 0.8403 - val_loss: 0.4466 - val_accuracy: 0.8571
Epoch 8/30
5/5 [==========

Test

In [ ]:
test_loss, test_accuracy = model.evaluate(test_X, test_y)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")

2/2 [==============================] - 0s 11ms/step - loss: 0.0319 - accuracy: 1.0000
Test loss: 0.03188816085457802, Test accuracy: 1.0


In [ ]:
model.save("my_model.h5")

# Load model
model = load_model("my_model.h5")

# Convert to tflite format and save
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS, 
    tf.lite.OpsSet.SELECT_TF_OPS
]
converter._experimental_lower_tensor_list_ops = False
tflite_model = converter.convert()

# Save the model to disk
open("gesture_model.tflite", "wb").write(tflite_model)

# Check the size of the tflite model file
import os
basic_model_size = os.path.getsize("gesture_model.tflite")
print("Model is %d bytes" % basic_model_size)


Model is 49744 bytes


In [ ]:
!echo "const unsigned char model[] = {" > /content/model.h
!cat gesture_model.tflite | xxd -i      >> /content/model.h
!echo "};"                              >> /content/model.h

import os
model_h_size = os.path.getsize("model.h")
print(f"Header file, model.h, is {model_h_size:,} bytes.")
# print("\nOpen the side panel (refresh if needed). Double click model.h to download the file.")

Header file, model.h, is 306,790 bytes.


In [ ]:
files.download('model.h')
print('model.h Downloaded')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

model.h Downloaded
